In [1]:
DEEPSEEK_API_KEY = 'sk-006151880202441eb4718f3f712fd48a'

# 1. 处理训练数据
## 1.1 数据读取

In [2]:
import re

In [3]:
# 读取参考数据集
import json,time
from functools import reduce
novel_data = []
with open('after12_final_4.json', 'r',encoding='utf-8') as file:
    # for line in file:
    #     novel_data.append(json.loads(line))
    novel_json_data = json.load(file)

In [4]:
# 查看数据集中所有小说的名字和字数
# for i in novel_data:
for i in novel_json_data:
    print(f"《{i['name']}》的字数为： {len(i['text'])} 字")
    

《案例》的字数为： 18 字
《特产》的字数为： 166 字
《三全其美》的字数为： 146 字
《购物狂》的字数为： 95 字
《失败之处》的字数为： 82 字
《找不开》的字数为： 144 字
《新品调料》的字数为： 123 字
《特异功能》的字数为： 187 字
《误解》的字数为： 168 字
《圆规的用途》的字数为： 105 字
《好消息和坏消息》的字数为： 106 字
《怎么照》的字数为： 109 字
《菜鸟学化妆》的字数为： 137 字
《最后的玫瑰》的字数为： 168 字
《有妈妈呢》的字数为： 144 字
《窗外的老乡》的字数为： 1918 字
《完美的爱》的字数为： 275 字
《贫富之间》的字数为： 320 字
《你家丢东西了吗》的字数为： 413 字
《团圆》的字数为： 403 字
《不看电视的夫妻》的字数为： 2375 字
《炸桥》的字数为： 2014 字
《改名字》的字数为： 1421 字
《平安路》的字数为： 2627 字
《换房游》的字数为： 3036 字
《良心买卖》的字数为： 2430 字
《恩比天高》的字数为： 2594 字
《消失的棋子》的字数为： 2816 字
《老父亲的丧事》的字数为： 2065 字
《绝对限量版》的字数为： 1313 字
《压力》的字数为： 1 字
《平地风波》的字数为： 716 字
《一字之差》的字数为： 1226 字
《镇店之宝》的字数为： 1577 字
《双雄记》的字数为： 3561 字
《大树的报复》的字数为： 2815 字
《喊鹊桥》的字数为： 1 字
《鹊桥传说》的字数为： 1047 字
《一喊成桥》的字数为： 871 字
《误坠桥下》的字数为： 1060 字
《仙姑真相》的字数为： 549 字
《最好的味道》的字数为： 10010 字
《文明的标志》的字数为： 273 字
《欢迎逃单拉面馆》的字数为： 311 字
《死角再生》的字数为： 406 字
《皮鞋》的字数为： 1295 字
《阿P写网文》的字数为： 1521 字
《吃鱼的学问》的字数为： 1 字
《雷人语录》的字数为： 1 字
《网络三国》的字数为： 349 字
《工资单里的猫腻》的字数为： 1454 字
《人类的朋友》的字数为： 625 字
《穿越之后》的字数为： 311 字
《照顾你面子》的字数为： 600 字
《抢冠军》的字数为

In [5]:
novel_json_data_more_400 = []
for n_obj in novel_json_data:
    if len(n_obj['text']) > 400:
        novel_json_data_more_400.append(n_obj)
        print(f"《{n_obj['name']}》的字数为： {len(n_obj['text'])} 字")

《窗外的老乡》的字数为： 1918 字
《你家丢东西了吗》的字数为： 413 字
《团圆》的字数为： 403 字
《不看电视的夫妻》的字数为： 2375 字
《炸桥》的字数为： 2014 字
《改名字》的字数为： 1421 字
《平安路》的字数为： 2627 字
《换房游》的字数为： 3036 字
《良心买卖》的字数为： 2430 字
《恩比天高》的字数为： 2594 字
《消失的棋子》的字数为： 2816 字
《老父亲的丧事》的字数为： 2065 字
《绝对限量版》的字数为： 1313 字
《平地风波》的字数为： 716 字
《一字之差》的字数为： 1226 字
《镇店之宝》的字数为： 1577 字
《双雄记》的字数为： 3561 字
《大树的报复》的字数为： 2815 字
《鹊桥传说》的字数为： 1047 字
《一喊成桥》的字数为： 871 字
《误坠桥下》的字数为： 1060 字
《仙姑真相》的字数为： 549 字
《最好的味道》的字数为： 10010 字
《死角再生》的字数为： 406 字
《皮鞋》的字数为： 1295 字
《阿P写网文》的字数为： 1521 字
《工资单里的猫腻》的字数为： 1454 字
《人类的朋友》的字数为： 625 字
《照顾你面子》的字数为： 600 字
《永恒之城——罗马》的字数为： 448 字
《梦中的城——曼谷》的字数为： 810 字
《必有内情》的字数为： 1043 字
《初露端倪》的字数为： 1092 字
《原来如此》的字数为： 1488 字
《买个李鬼》的字数为： 971 字
《步步惊心》的字数为： 1485 字
《原来如此》的字数为： 968 字
《给局长发稿费》的字数为： 1933 字
《包你满意》的字数为： 1423 字
《爱心行动》的字数为： 1864 字
《为了兄弟情》的字数为： 2456 字
《老板心太软》的字数为： 2851 字
《知县找护卫》的字数为： 1983 字
《阿P占车位》的字数为： 2172 字
《想做富二代》的字数为： 2417 字
《真正的英雄》的字数为： 574 字
《相片姻缘》的字数为： 522 字
《哪里是家》的字数为： 1411 字
《穿长裙子的女教师》的字数为： 2008 字
《劫后余生》的字数为： 835 字
《前尘往事》的字数为： 1644 字
《拨云见日》的字数为： 852 

In [6]:
len(novel_json_data_more_400)

719

## 1.2 数据分块

In [7]:
# 安装必要的库
# !pip install jieba loguru openai -i https://pypi.tuna.tsinghua.edu.cn/simple

## 1.3 大模型构建数据集

In [8]:
import json
from tqdm import tqdm
import time
import os
from openai import OpenAI

In [9]:
# 使用deepseek-chat api给段落打标签的接口
def get_response(text):
    client = OpenAI(
        api_key=DEEPSEEK_API_KEY,  # 如果您没有配置环境变量，请在此处用您的API Key进行替换
        base_url="https://api.deepseek.com",  # 填写DashScope SDK的base_url
    )
    completion = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {
                'role': 'system', 
                'content': '总结user提交的内容。用一句不超过50字的话总结这段小说的情节。仅回答总结，不需要添加其他内容。'
            },
            {
                'role': 'user', 
                'content': text
            }
        ])
    
    return completion.choices[0].message.content

In [10]:
# 设置容错机制，可最多重试 5 次，如果失败记录错误日志
def get_summary_with_retry(text):
    max_retries = 5
    retry_delay = 60  # in seconds
    attempts = 0
    while attempts < max_retries:
        try:
            return get_response(text)
        except Exception as e:
            attempts += 1
            if attempts < max_retries:
                # logger.warning(f"Attempt {attempts} failed for text: {text}. Retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)
            else:
                # logger.error(f"All {max_retries} attempts failed for text: {text}. Error: {e}")
                raise

In [11]:
# 创建文件夹
os.makedirs('data', exist_ok=True)
os.makedirs('output', exist_ok=True)
os.makedirs('dataset', exist_ok=True)

In [12]:
# 批量给指定的小说打标签的接口函数
def build_dataset(novel,texts):
    instruction_prompt = "你是一个熟读各类小说的专家，请你根据要求写一段800字左右的小说。"
    dataset = []
    dataset_error = []
    for text in tqdm(texts, desc=f"Processing {novel}", total=len(texts)):
        try:
            summary = get_summary_with_retry(text)
            print(summary)
            dataset.append({
                "instruction": instruction_prompt,
                "input": summary,
                "output": text
            })
        except Exception as e:
            dataset_error.append(text)
            # logger.error(f"Failed to process text: {text}. Error: {e}")
    
    with open(f"./data/{novel}.json", "w",encoding='utf-8') as f:
        f.write(json.dumps(dataset, ensure_ascii=False, indent=4))

    with open(f"./data/{novel}_error.txt", "w",encoding='utf-8') as f:
        f.write(json.dumps(dataset_error, ensure_ascii=False, indent=4))
    return dataset

In [13]:
# !pip install logger -i https://pypi.mirrors.ustc.edu.cn/simple

In [14]:
import jieba
# import 

In [15]:
# 拆分为 800 字一段的段落
# type(novel_json_data_more_400)
for n_obj in novel_json_data_more_400:
    data = n_obj["text"]
    story_name = n_obj["name"]
    print(len(data))

    # 利用jieba进行句子切分
    sentences = []

    for sentence in data.split('。'):  # 使用句号作为切分符
        sentences.append(sentence)

    # 将句子合并成800字一段的段落
    paragraphs = []
    current_paragraph = ''
    for sentence in sentences:
        if len(current_paragraph) + len(sentence) <= 800:
            current_paragraph += sentence+'。'
        else:
            paragraphs.append(current_paragraph.strip())
            current_paragraph = sentence

    # 将最后一段加入到段落列表中
    if current_paragraph:
        paragraphs.append(current_paragraph.strip())

    # 打印切分后的段落
    # for idx, paragraph in enumerate(paragraphs):
    #     print(f'段落 {idx + 1}: {paragraph}')
    print(len(paragraphs))
    # 开始给段落打标签
    dataset = build_dataset(story_name,paragraphs[:len(paragraphs)])

1918
3


Processing 窗外的老乡:  33%|███▎      | 1/3 [00:05<00:11,  5.61s/it]

夏天周末，作者和儿子在家，看到窗外粉刷工人，儿子兴奋称其为“蜘蛛侠叔叔”。工人是河南老乡，两人用家乡话交流，工人对作者在上海的生活表示赞赏，但无意中留下了脚印。


Processing 窗外的老乡:  67%|██████▋   | 2/3 [00:10<00:05,  5.02s/it]

一位河南老乡在高空作业时与主人公聊天，询问关于船舶公司的事情，并讨论了年龄和收入，最后在路边相遇时老乡热情地分享西瓜。


...

幼儿园绘画比赛中，蒂姆以独特方式用尿在地上画出一匹栩栩如生的马，令老师目瞪口呆。
590
1


Processing 酒鬼挂横幅: 100%|██████████| 1/1 [00:05<00:00,  5.03s/it]


宣传委员小军让打杂工老李在横幅上贴“三八”庆祝妇女节，但老李因数字破损而贴了“廿四”，解释为三乘八等于二十四。
572
1


Processing 终于拿了二等奖: 100%|██████████| 1/1 [00:04<00:00,  4.34s/it]

学校参加小学生演讲比赛，黄老师带队获得“二等奖”，但因比赛奖项调整，实际上仍是末等奖，高校长误以为改写历史，最终仍是“末奖专业户”。


In [16]:
# dataset = build_dataset(story_name,paragraphs[:len(paragraphs)])

## 2.4 数据合并
由于采用多个小说数据进行模型微调，所以将data下的所有json数据文件合并成dataset文件夹下的`merged_story.json`文件

In [17]:
import json
import os

# 设置文件夹路径
directory_path = './data'

# 初始化一个空列表，用于存储合并后的数据
merged_data = []

# 遍历文件夹下的所有文件
for filename in os.listdir(directory_path):
    # 检查文件扩展名是否为.json
    if filename.endswith('.json'):
        # 构建文件的完整路径
        file_path = os.path.join(directory_path, filename)
        # 打开并读取JSON文件
        with open(file_path, 'r', encoding='utf-8') as file:
            # 加载JSON内容到变量
            data = json.load(file)
            # 将当前文件的数据添加到合并列表中
            merged_data.extend(data)

# 将合并后的数据转换为JSON格式
merged_json = json.dumps(merged_data, ensure_ascii=False, indent=4)

# 可以选择将合并后的数据写入到一个新的JSON文件中
output_file_path = './dataset/merged_story.json'
with open(output_file_path, 'w', encoding='utf-8') as file:
    file.write(merged_json)

# 或者直接输出到控制台
# print(merged_json)

## 2.5 处理数据集

In [18]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig

ModuleNotFoundError: No module named 'transformers'

In [ ]:
# 将JSON文件转换为CSV文件
df = pd.read_json('after12_final_4.json')
# df = pd.read_json('./data/story/呼啸山庄.json')
ds = Dataset.from_pandas(df)

In [ ]:
print(ds[:3])  # 查看一条数据
print(len(ds)) # 呼啸山庄.json, 总共84条微调指令数据, 如果采用merged_story.json，共9990条